# Project Part 3: Adversarial, Transferability and Robustification



We recommand you to use Google Colab to edit and run this notebook. You can also install jupyter on your own computer.

In [3]:
import torch
import numpy as np
from sklearn.datasets import fetch_openml
from torch import nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

## 0. Prepare data

You can familiarise yourself with MNIST, a small size dataset, on its Wikipedia article [https://en.wikipedia.org/wiki/MNIST_database](https://en.wikipedia.org/wiki/MNIST_database). MNIST is composed of 28x28 grayscaled images of handwritten digits. This is a classification task with 10 classes (10 digits).

In [4]:
# Data Loading
mnist = fetch_openml('mnist_784', as_frame=False, cache=True)

In [5]:
x = mnist["data"]
y = mnist["target"]

In [6]:
# Data exploration
print(f"Shape of x: {x.shape}")
print(f"Min, max x: {x.min(), x.max()}")
print(f"Shape of y: {y.shape}")
print(f"Classes in y: {np.unique(y)}")

Shape of x: (70000, 784)
Min, max x: (0, 255)
Shape of y: (70000,)
Classes in y: ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9']


In [7]:
# Split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42, stratify=y_train, shuffle=True)

In [8]:
# # Preprocessing
# # x = torch.from_numpy(x.astype(float)).float()
# # y = torch.from_numpy(y.astype(int)).type(torch.LongTensor)
# x = torch.tensor(x, dtype=torch.float32)
# y = torch.tensor(y.astype(int), dtype=torch.long)

# # Shape
# x = x.reshape(-1, 1, 28, 28)
# # Scaler
# x = (x - x.min()) / (x.max() - x.min())

In [9]:
def prepare_data(test_size=0.2, val_size=0.2, random_state=42):
    """
    Loads and prepares the MNIST dataset.
    
    Parameters:
    - test_size (float): Proportion of the dataset to include in the test split.
    - val_size (float): Proportion of the train dataset to include in the validation split.
    - random_state (int): Random seed for reproducibility.
    
    Returns:
    - x_train, x_val, x_test (np.ndarray): Training, validation, and testing feature sets.
    - y_train, y_val, y_test (np.ndarray): Corresponding labels.
    """
    # Load the MNIST dataset
    mnist = fetch_openml('mnist_784', as_frame=False, cache=True)
    x = mnist["data"]
    y = mnist["target"]
    
    # Data exploration
    print(f"Shape of x: {x.shape}")
    print(f"Min, max x: {x.min(), x.max()}")
    print(f"Shape of y: {y.shape}")
    print(f"Classes in y: {np.unique(y)}")
    
    # Train-test split
    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=test_size, random_state=random_state, stratify=y, shuffle=True
    )
    
    # Train-validation split
    x_train, x_val, y_train, y_val = train_test_split(
        x_train, y_train, test_size=val_size, random_state=random_state, stratify=y_train, shuffle=True
    )
    
    return x_train, x_val, x_test, y_train, y_val, y_test

# Prepare data
x_train, x_val, x_test, y_train, y_val, y_test = prepare_data()

Shape of x: (70000, 784)
Min, max x: (0, 255)
Shape of y: (70000,)
Classes in y: ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9']


In [10]:
def convert_to_tensor(data, dtype=torch.float32, reshape=None, normalize=False):
    """
    Convert a NumPy array to a PyTorch tensor with optional reshaping and normalization.
    
    Parameters:
    - data (np.ndarray): Input NumPy array.
    - dtype (torch.dtype): Desired PyTorch data type.
    - reshape (tuple): New shape for the tensor (optional).
    - normalize (bool): Whether to normalize data to [0, 1].
    
    Returns:
    - torch.Tensor: Converted PyTorch tensor.
    """
    tensor = torch.tensor(data, dtype=dtype)
    if reshape:
        tensor = tensor.reshape(reshape)
    if normalize:
        tensor = (tensor - tensor.min()) / (tensor.max() - tensor.min())
    return tensor

def prepare_tensors(x_train, x_val, x_test, y_train, y_val, y_test):
    """
    Prepares PyTorch tensors for the dataset by reshaping and normalizing.

    Parameters:
    - x_train, x_val, x_test (np.ndarray): Feature datasets.
    - y_train, y_val, y_test (np.ndarray): Label datasets.

    Returns:
    - Tensors for training, validation, and testing datasets (features and labels).
    """
    x_train_tensor = convert_to_tensor(x_train, dtype=torch.float32, reshape=(-1, 1, 28, 28), normalize=True)
    x_val_tensor = convert_to_tensor(x_val, dtype=torch.float32, reshape=(-1, 1, 28, 28), normalize=True)
    x_test_tensor = convert_to_tensor(x_test, dtype=torch.float32, reshape=(-1, 1, 28, 28), normalize=True)

    y_train_tensor = convert_to_tensor(y_train.astype(int), dtype=torch.long)
    y_val_tensor = convert_to_tensor(y_val.astype(int), dtype=torch.long)
    y_test_tensor = convert_to_tensor(y_test.astype(int), dtype=torch.long)
    
    return x_train_tensor, x_val_tensor, x_test_tensor, y_train_tensor, y_val_tensor, y_test_tensor

# Prepare tensors
x_train_tensor, x_val_tensor, x_test_tensor, y_train_tensor, y_val_tensor, y_test_tensor = prepare_tensors(
    x_train, x_val, x_test, y_train, y_val, y_test
)

# Verify the results
print(f"x_train_tensor shape: {x_train_tensor.shape}")
print(f"y_train_tensor shape: {y_train_tensor.shape}")

x_train_tensor shape: torch.Size([44800, 1, 28, 28])
y_train_tensor shape: torch.Size([44800])


## 1. Adversarial examples

The goal of this first part is to generate adversarial examples on a simple dataset called MNIST. MNIST is a dataset of 28x28 black and white images that represents hand-written digits, and their associate label 0,1,...,9.

You can use the following ressource to help you [https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html#](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html#).


1. Train a Neural Network using the PyTorch library.

The architecture of the models and the training hyper-parameters are given below.
We recommend using these parameters, the SGD optimizer and the Cross Entropy loss.


In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

In [12]:
learning_rate = 0.001
momentum = 0.9
epochs = 10
batch_size = 64

In [13]:
# learning_rate = 0.01  # Slightly increase learning rate for faster convergence
# momentum = 0.9  # Keep momentum unchanged
# epochs = 2  # Reduce epochs to quickly validate the code
# batch_size = 8  # Reduce batch size to fit in limited memory

In [14]:
model_0 = Net()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_0.to(device)

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [15]:
optimizer = torch.optim.SGD(model_0.parameters(), lr=learning_rate, momentum=momentum)
loss_func = nn.CrossEntropyLoss()

In [16]:
def train_loop(dataloader, model, loss_fn, optimizer, batch_size):
    size = len(dataloader.dataset)
    model.train()  # Set the model to training mode
    total_loss = 0.0  # Track total loss for the epoch
    
    for batch, (X, y) in tqdm(enumerate(dataloader), total=int(size / batch_size), desc="Training"):
        # Move data and labels to the appropriate device
        X, y = X.to(next(model.parameters()).device), y.to(next(model.parameters()).device)

        # Compute prediction and loss
        ## YOUR CODE HERE:
        preds = model(X)  # Forward pass
        loss = loss_fn(preds, y)  # Compute the loss

        # Backpropagation
        ## YOUR CODE HERE:
        optimizer.zero_grad()  # Reset gradients to zero
        loss.backward()  # Backward pass: compute gradients
        optimizer.step()  # Update model parameters

        # Track the loss for this batch
        total_loss += loss.item()
    
    # Return the average loss for the epoch
    return total_loss / len(dataloader)

In [20]:
# # ## GIVEN, to evaluate the progress of the training at each epoch
# def val_loop(dataloader, model, loss_fn, epoch_i):
#     size = len(dataloader.dataset)
#     num_batches = len(dataloader)
#     test_loss, correct = 0, 0

#     # Set model to evaluation mode
#     model.eval()

#     with torch.no_grad():
#         for X, y in dataloader:
#             # Move data and labels to the same device as the model
#             X, y = X.to(next(model.parameters()).device), y.to(next(model.parameters()).device)

#             # Forward pass
#             pred = model(X)

#             # Compute loss and accuracy
#             test_loss += loss_fn(pred, y).item()
#             correct += (pred.argmax(1) == y).type(torch.float).sum().item()

#     # Compute average loss and accuracy
#     test_loss /= num_batches
#     correct /= size

#     # Print metrics
#     print(f"Epoch {epoch_i}, Val Error: Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")

def val_loop(dataloader, model, loss_fn, epoch_i, verbose=True):
    """
    Evaluate the model on the validation dataset.

    Parameters:
    - dataloader (DataLoader): DataLoader for the validation dataset.
    - model (torch.nn.Module): Model to evaluate.
    - loss_fn (torch.nn.Module): Loss function.
    - epoch_i (int): Current epoch number.
    - verbose (bool): Whether to print validation metrics.

    Returns:
    - float: Average validation loss.
    - float: Validation accuracy (percentage).
    """
    # Validate inputs
    if not isinstance(dataloader, DataLoader):
        raise ValueError("dataloader must be an instance of torch.utils.data.DataLoader")

    # Initialize evaluation
    device = next(model.parameters()).device
    model.eval()  # Set model to evaluation mode
    val_loss, num_correct = 0.0, 0
    total_samples = len(dataloader.dataset)
    num_batches = len(dataloader)

    # Evaluate in batches
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            # Forward pass
            preds = model(X)

            # Compute loss and accuracy
            val_loss += loss_fn(preds, y).item()
            num_correct += (preds.argmax(1) == y).type(torch.float).sum().item()

    # Compute average loss and accuracy
    val_loss /= num_batches
    accuracy = (num_correct / total_samples) * 100

    # Print metrics if verbose
    if verbose:
        print(f"Epoch {epoch_i}, Val Error: Accuracy: {accuracy:>0.1f}%, Avg loss: {val_loss:>8f}")

    return val_loss, accuracy

In [21]:
# def train_model(model, x_train, y_train, x_val, y_val, optimizer, batch_size, loss_func, epochs):
#     # Data processing
#     train_dataset = TensorDataset(x_train, y_train)
#     train_loader = DataLoader(
#         dataset=train_dataset,
#         batch_size=batch_size,
#         shuffle=True,
#         num_workers=2,
#     )
#     val_dataset = TensorDataset(x_val, y_val)
#     val_loader = DataLoader(
#         dataset=val_dataset,
#         batch_size=batch_size,
#         shuffle=False,
#         num_workers=2,
#     )

#     for epoch in range(1, epochs + 1):
#         print(f"Epoch {epoch}/{epochs}")
#         model.train()  # Set model to training mode

#         # Training loop
#         train_loss = 0.0
#         for X, y in tqdm(train_loader, desc="Training"):
#             # Move data to the same device as the model
#             X, y = X.to(next(model.parameters()).device), y.to(next(model.parameters()).device)

#             # Forward pass
#             preds = model(X)
#             loss = loss_func(preds, y)

#             # Backward pass and optimization
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#             train_loss += loss.item()

#         # Average train loss for this epoch
#         avg_train_loss = train_loss / len(train_loader)
#         print(f"Train Loss: {avg_train_loss:.4f}")

#         # Validation loop
#         model.eval()  # Set model to evaluation mode
#         val_loss = 0.0
#         correct = 0
#         total = 0
#         with torch.no_grad():
#             for X, y in val_loader:
#                 X, y = X.to(next(model.parameters()).device), y.to(next(model.parameters()).device)

#                 # Forward pass
#                 preds = model(X)
#                 loss = loss_func(preds, y)
#                 val_loss += loss.item()

#                 # Compute accuracy
#                 pred_classes = preds.argmax(dim=1)
#                 correct += (pred_classes == y).sum().item()
#                 total += y.size(0)

#         # Average validation loss and accuracy
#         avg_val_loss = val_loss / len(val_loader)
#         val_accuracy = correct / total * 100
#         print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")


def train_model(model, x_train, y_train, x_val, y_val, optimizer, batch_size, loss_func, epochs):
    """
    Train the model and evaluate on validation data after each epoch.

    Parameters:
    - model (torch.nn.Module): The model to train.
    - x_train, y_train (torch.Tensor): Training features and labels.
    - x_val, y_val (torch.Tensor): Validation features and labels.
    - optimizer (torch.optim.Optimizer): Optimizer for parameter updates.
    - batch_size (int): Batch size for training.
    - loss_func (torch.nn.Module): Loss function.
    - epochs (int): Number of epochs to train.

    Returns:
    - dict: Dictionary containing training and validation metrics.
    """
    # Prepare data loaders
    train_dataset = TensorDataset(x_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_dataset = TensorDataset(x_val, y_val)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    # Track metrics
    history = {"train_loss": [], "val_loss": [], "val_accuracy": []}

    # Training and validation for each epoch
    for epoch in range(1, epochs + 1):
        print(f"Epoch {epoch}/{epochs}")

        # Training loop
        train_loss = train_loop(train_loader, model, loss_func, optimizer, batch_size)
        print(f"Train Loss: {train_loss:.4f}")

        # Validation loop
        val_loss, val_accuracy = val_loop(val_loader, model, loss_func, epoch_i=epoch, verbose=False)
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

        # Log metrics
        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["val_accuracy"].append(val_accuracy)

    return history

In [22]:
## YOUR CODE HERE: train the model using the training function you just implemented.
train_model(
    model=model_0, 
    x_train=x_train_tensor, 
    y_train=y_train_tensor, 
    x_val=x_val_tensor, 
    y_val=y_val_tensor, 
    optimizer=optimizer, 
    batch_size=batch_size, 
    loss_func=loss_func, 
    epochs=epochs
)

Epoch 1/10


Training: 100%|██████████| 700/700 [00:26<00:00, 26.21it/s]

Train Loss: 1.2921


Validation Loss: 0.4926, Validation Accuracy: 87.29%
Epoch 2/10


Training: 100%|██████████| 700/700 [00:28<00:00, 24.32it/s]

...

Validation Loss: 0.1368, Validation Accuracy: 95.78%
Epoch 10/10


Training: 100%|██████████| 700/700 [00:13<00:00, 53.31it/s] 

Train Loss: 0.3113


Validation Loss: 0.1265, Validation Accuracy: 96.24%


{'train_loss': [1.2921131033556803,
  0.702658565725599,
  0.5554901785935674,
  0.4844007088669709,
  0.4331578298977443,
  0.3966901205480099,
  0.3643378542363644,
  0.34076620885304043,
  0.32424190448863166,
  0.31129711951528277],
 'val_loss': [0.4926051558767046,
  0.32453633499997003,
  0.26705293084893905,
  0.21976818195411138,
  0.188719513629164,
  0.17018323852547576,
  0.15517446294426918,
  0.14675663988505092,
  0.13678530863353183,
  0.12647326651428428],
 'val_accuracy': [87.28571428571429,
  91.46428571428571,
  92.66964285714285,
  93.70535714285714,
  94.51785714285714,
  94.97321428571428,
  95.33928571428572,
  95.80357142857143,
  95.77678571428572,
  96.24107142857142]}

2. Evaluate clean accuracy of the Neural Network using a test set that has not been used for training.

In [23]:
# Set model into evaluation mode
model_0.eval()

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [24]:
## YOUR CODE HERE: Evaluate model accuracy

accuracy = None
print(f"Clean accuracy of the model is {accuracy}.")

Clean accuracy of the model is None.


In [25]:
# Wrap the test dataset in a DataLoader
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=2000, shuffle=False)

# Initialize variables for tracking correct predictions and total samples
correct = 0
total = 0

# Evaluate the model
with torch.no_grad():  # Disable gradient calculations
    for X, y in test_loader:  # Iterate through the test DataLoader
        # Move data and labels to the appropriate device
        X, y = X.to(device), y.to(device)

        # Forward pass to compute predictions
        preds = model_0(X)

        # Get predicted class labels
        pred_classes = preds.argmax(dim=1)

        # Count correct predictions
        correct += (pred_classes == y).sum().item()

        # Track total samples
        total += y.size(0)

# Calculate accuracy
accuracy = correct / total * 100

# Print the clean accuracy
print(f"Clean accuracy of the model is {accuracy:.2f}%.")

Clean accuracy of the model is 96.16%.


3. Implement and execute the PGD attack on 1000 examples of the testing set. The hyperparameters of PGD are given below.
The perturbation is bounded by a maximum L-infinity norm, called epsilon (eps), which means that each pixel can be perturbed between -eps and +eps. We initialy set the maximum perturbation to eps = 32/255. For simplicity, you can set the step size alpha = epsilon / 10, and run PGD with only one random restart.

You can find the description of PGD in the paper [https://arxiv.org/abs/1706.06083](https://arxiv.org/abs/1706.06083) and an example of another adversarial attack on the PyTorch documentation [https://pytorch.org/tutorials/beginner/fgsm_tutorial.html](https://pytorch.org/tutorials/beginner/fgsm_tutorial.html).
Tips: use the F.cross_entropy loss during the attack.


In [26]:
n_examples = 1000
eps = 32/255
n_iter = 50
alpha = eps / 10

In [27]:
## YOUR CODE HERE: Generate adversarial examples


4. Show the robust accuracy of model_0, that is the accuracy of the model on the adversarial examples.

In [28]:
## YOUR CODE HERE: Evaluate model robust accuracy


5. Show the impact of the maximum perturbation allowed (denoted epsilon).

In [29]:
eps = [8/255, 16/255, 32/255, 64/255]
alpha = [e/10 for e in eps]

In [30]:
## YOUR CODE HERE: compute the adversarial examples for each provided epsilon
## (maximum l-infinity norm of the perturbation), and compute the associated robust accuracy
## Use a graph to display your result. You may use the [Matplotlib] (https://matplotlib.org/stable/index.html).

6. Using matplotlib, plot 10 adversarial examples, along with their corresponding original images. Choose one original image classified per class (the 10 class should be represented). For each image (adversarial and original), add on the plot the predicted class of the image.


In [31]:
## YOUR CODE HERE

**Question**: Please comment your results of this section.

**ANSWER HERE**


## 2. Transferability

In this section we will see how adversarial examples generated on one model can be adversarial on another model using a different architecture.
Let suppose a second model which parameters are unknown. For instance, it could be a model deploy on a cloud platform. We will use the examples generated in Section 1 on model_0 to fool this new model denoted model_1.
We say that model_0 is a surrogate for model_1.

1. Define a neural network architecture for MNIST different than the one used in Section 1.

In [32]:
## GIVEN
class FullyConnectedNetwork(nn.Module):
    def __init__(self):
        super(FullyConnectedNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

2. Train the neural network model_1 with the same hyperparameters as model_0


In [33]:
## YOUR CODE HERE
model_1 = None
optimizer = None  # create a new optimizer object when you train a new model

3. What is the ratio of successful adversarial examples on model_0 that transfers to model_1 (ie. that are also adversarial for model_1)?


In [34]:
model_1.eval()
## YOUR CODE HERE

AttributeError: 'NoneType' object has no attribute 'eval'

What do you conclude about the robustness of the model? Can [secrecy](https://en.wikipedia.org/wiki/Security_through_obscurity) defend a model?

**ANSWER HERE**

## 3. Use adversarial training to robustify the model

Adversarial training is a common method to robustify models to adversarial examples as described in this paper [https://arxiv.org/abs/1706.06083](https://arxiv.org/abs/1706.06083). In this section you should update the training loop such that 3/4 of the batch is used for training while the remaining forth is first perturbed with PGD and then used for training. You can limit the number of iterations of PDG to 10. Use model_0 architecture from Section 1 in this section.

1. Train model_robust using adversarial training. You may want to run it for additional epoch (x2) to reach a similar clean accuracy.

In [ ]:
n_iter = 10  # less iterations to accelerate training. But once trained, we will still evaluate the robust accuracy on more iterations for a more powerful attack.
eps = 32/255
alpha = eps / 5
model_robust = Net()  # newly initialized NN

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, batch_size):
    size = len(dataloader.dataset)
    adv_size = int(batch_size/4)
    for batch, (X, y) in tqdm(enumerate(dataloader), total=int(size/batch_size)):

        # Generate adversarial examples for a forth of the data

        model.eval()
        ## YOUR CODE HERE
        model.train()

        # Compute prediction and loss

        ## YOUR CODE HERE:

        # Backpropagation

        ## YOUR CODE HERE:



In [ ]:
## YOUR CODE HERE: The rest of training implementation is unchanged.
## Do not reuse the same optimizer object!!!

2. Compare the robust accuracies of model_0 and model_robust using the same hyperparameters of PGD for different eps size, use a graph to show your results.

In [ ]:
n_examples = 1000
n_iter = 50
eps = [8/255, 16/255, 32/255, 64/255]
alpha = [e/10 for e in eps]

In [ ]:
## YOUR CODE HERE

**Questions**: Please comment your results. Does adversarial training appears to be a valid defense? Please develop threads to validity of the robust accuracy evaluation carried out here. What could be done to improve the evaluation of the robustness of the model?

**ANSWER HERE**